In [ ]:
import torch
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 9.4 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 6.5 MB 53.2 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 68.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install datasets
# @article{kennedy2020constructing,
#   title={Constructing interval variables via faceted Rasch measurement and multitask deep learning: a hate speech application},
#   author={Kennedy, Chris J and Bacon, Geoff and Sahn, Alexander and von Vacano, Claudia},
#   journal={arXiv preprint arXiv:2009.10277},
#   year={2020}
# }
# Reference : https://raphaelb.org/posts/freezing-bert/

     |████████████████████████████████| 325 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 60.2 MB/s 
     |████████████████████████████████| 212 kB 62.3 MB/s 
     |████████████████████████████████| 136 kB 69.4 MB/s 
     |████████████████████████████████| 127 kB 57.7 MB/s 
     |████████████████████████████████| 271 kB 23.4 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 
     |████████████████████████████████| 144 kB 61.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import datasets
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   


Using custom data configuration ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df = dataset['train'].to_pandas()
data = df[["text", "hate_speech_score", "hatespeech"]]

In [ ]:
data.hatespeech.value_counts()

0.0    80624
2.0    46021
1.0     8911
Name: hatespeech, dtype: int64

In [ ]:
# data[data.hate_speech_score==0.00]

In [ ]:
data.hatespeech = data.hatespeech.apply(lambda x : 0 if x == 0.0 else 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
data.hatespeech.value_counts()

0    80624
1    54932
Name: hatespeech, dtype: int64

In [ ]:
data.rename(columns={"hatespeech":"labels"}, inplace=True)
data.drop(columns="hate_speech_score", inplace=True)
# train_x = data["text"]
# train_y = data["hatespeech"]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# from sklearn.model_selection import train_test_split
# train_ds, test_ds = train_test_split(data, test_size=0.1, random_state=42)


In [ ]:
train_ds = data

In [ ]:
type(train_ds)

pandas.core.frame.DataFrame

In [ ]:
# test_ds = pd.read_json("/content/drive/MyDrive/Colab Notebooks/test_seen.jsonl",lines=True)

test_ds = pd.read_json("/content/drive/MyDrive/522-project/dev_unseen.jsonl",lines=True)


In [ ]:
train_ds.head()

,text,labels
0,Yes indeed. She sort of reminds me of the elde...,0
1,The trans women reading this tweet right now i...,0
2,Question: These 4 broads who criticize America...,1
3,It is about time for all illegals to go back t...,0
4,For starters bend over the one in pink and kic...,1


In [ ]:
test_ds.head()

,id,img,label,text
0,76432,img/76432.png,0,you thinking what i'm thinking?
1,14270,img/14270.png,0,a brilliant mind
2,56947,img/56947.png,0,"pro gamer 6.000.000 kills, 1 death"
3,35174,img/35174.png,0,lets end poaching
4,39264,img/39264.png,0,my wife called me a good husband thats a huge ...


In [ ]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_ds)

In [ ]:
test_ds = Dataset.from_pandas(test_ds)

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModel
task='hate'
model_str = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(model_str)
model = AutoModelForSequenceClassification.from_pretrained(model_str, output_hidden_states =True)

#tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/roberta-base-nli-stsb-mean-tokens')

#model = RobertaForSequenceClassification.from_pretrained('roberta-base',output_hidden_states =True)
#tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

Downloading:   0%|          | 0.00/588 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_ds = train_ds.map(tokenization, batched = True, batch_size = len(train_ds)) # TODO : add more training data here.
test_ds = test_ds.map(tokenization, batched = True, batch_size = len(test_ds))


  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
type(train_ds), type(test_ds)

(datasets.arrow_dataset.Dataset, datasets.arrow_dataset.Dataset)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import math

In [ ]:
import numpy as np

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = np.argmax(np.array(preds), axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def get_random_seed():
    return int.from_bytes(os.urandom(4), "big")

def train(
    train_size: None,
    freeze_layer_count: int = 8,
):
    # get_split_path returns the path of a npy file containing sample indexes
    # to include in the training set
    #split_path = get_split_path(dataset_name, train_size) if train_size is not None else None
    args_dict = {
        "output_dir" : '/content/drive/MyDrive/retrain',
        "evaluation_strategy": "steps",
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 16,
        "learning_rate": 5e-5,
        "num_train_epochs": 10,
        "logging_first_step": True,
        "save_total_limit": 1,
        "fp16": True,
        "dataloader_num_workers": 1,
        "load_best_model_at_end": True,
        "metric_for_best_model": "accuracy",
        # we need to generate a random seed manually, as otherwise 
        # the same constant random seed is used during training for each run
        "seed": get_random_seed(),
    }
    #model = AutoModelForSequenceClassification.from_pretrained("roberta-base", return_dict=True)

    if freeze_layer_count:
	      # We freeze here the embeddings of the model
        for param in model.roberta.embeddings.parameters():
            param.requires_grad = False

        if freeze_layer_count != -1:
	          # if freeze_layer_count == -1, we only freeze the embedding layer
	          # otherwise we freeze the first `freeze_layer_count` encoder layers
            for layer in model.roberta.encoder.layer[:freeze_layer_count]:
                for param in layer.parameters():
                    param.requires_grad = False

    #tokenizer = AutoTokenizer.from_pretrained("roberta-base")
    # train_ds = get_dataset(tokenizer, dataset_name, split="train", split_path=split_path)
    # val_ds = get_dataset(tokenizer, dataset_name, split="validation")

    epoch_steps = len(train_ds) / args_dict["per_device_train_batch_size"]
    args_dict["warmup_steps"] = math.ceil(epoch_steps)  # 1 epoch
    args_dict["logging_steps"] = max(1, math.ceil(epoch_steps * 0.5))  # 0.5 epoch
    args_dict["save_steps"] = args_dict["logging_steps"]

    training_args = TrainingArguments(**args_dict)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    trainer.train()


In [ ]:
train(train_size=100)

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 135556
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 84730


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
4237,0.452800,1.207904,0.625926,0.114035,0.464286,0.065000
8474,0.420100,0.998506,0.622222,0.163934,0.454545,0.100000
12711,0.413900,0.896008,0.627778,0.114537,0.481481,0.065000


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, img, text. If id, img, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 540
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/retrain/checkpoint-4237
Configuration saved in /content/drive/MyDrive/retrain/checkpoint-4237/config.json
Model weights saved in /content/drive/MyDrive/retrain/checkpoint-4237/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/retrain/checkpoint-4237/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/retrain/checkpoint-4237/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/retrain/checkpoint-1] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenc

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
4237,0.452800,1.207904,0.625926,0.114035,0.464286,0.065000
8474,0.420100,0.998506,0.622222,0.163934,0.454545,0.100000
12711,0.413900,0.896008,0.627778,0.114537,0.481481,0.065000
16948,0.406000,0.851779,0.631481,0.138528,0.516129,0.080000
21185,0.393500,0.985119,0.631481,0.145923,0.515152,0.085000
25422,0.393300,1.009736,0.629630,0.236641,0.500000,0.155000
29659,0.378300,0.894818,0.618519,0.188976,0.444444,0.120000
33896,0.382400,0.937744,0.611111,0.285714,0.446809,0.210000
38133,0.366700,1.201996,0.631481,0.174274,0.512195,0.105000
42370,0.368100,1.196167,0.627778,0.137339,0.484848,0.080000


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, img, text. If id, img, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 540
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/retrain/checkpoint-16948
Configuration saved in /content/drive/MyDrive/retrain/checkpoint-16948/config.json
Model weights saved in /content/drive/MyDrive/retrain/checkpoint-16948/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/retrain/checkpoint-16948/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/retrain/checkpoint-16948/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/retrain/checkpoint-12711] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaF

In [ ]:
train_data = pd.read_json("/content/drive/MyDrive/522-project/train.jsonl",lines=True)
test_seen_data = pd.read_json("/content/drive/MyDrive/522-project/test_seen.jsonl",lines=True)
test_unseen_data = pd.read_json("/content/drive/MyDrive/522-project/test_unseen.jsonl",lines=True)
dev_unseen_data = pd.read_json("/content/drive/MyDrive/522-project/dev_unseen.jsonl",lines=True)

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoding_input=[]
last_token=[]
text=train_data ["text"].tolist()
id = train_data ["id"].tolist()

for i,j in zip(text,id):
 

  b = tokenizer(i, return_tensors='pt').to(device)
  # # c= tokenizer.tokenize(i)
  # d=tokenizer.encode(i, add_special_tokens=True)
  # #print(b.tokens)
  # print(b)
  # print(c)
  # print(d)
  output = model(**b)
  #output= model(**i)

  f=output[1][1].squeeze()[0].cpu()
  g = output[1][1].squeeze()[-1].cpu()
  encoding_input.append([j,f.tolist()])
  last_token.append([j,g.tolist()])








In [ ]:
final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/retrain_embedding_train.csv')
final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/retrain_embedding_1_train.csv')

In [ ]:
print(len(encoding_input[0][1]))
print(len(last_token[0][1]))

In [ ]:
final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/drive/MyDrive/522-project/retrain_embedding.csv')


In [ ]:
final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/drive/MyDrive/522-project/retrain_embedding_1.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/522-project/retrain_embedding.csv')

In [ ]:
# validation 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoding_input=[]
last_token=[]
text=dev_unseen_data ["text"].tolist()
id = dev_unseen_data ["id"].tolist()

for i,j in zip(text,id):
 

  b = tokenizer(i, return_tensors='pt').to(device)
  # # c= tokenizer.tokenize(i)
  # d=tokenizer.encode(i, add_special_tokens=True)
  # #print(b.tokens)
  # print(b)
  # print(c)
  # print(d)
  output = model(**b)
  #output= model(**i)

  f=output[1][1].squeeze()[0].cpu()
  g = output[1][1].squeeze()[-1].cpu()
  encoding_input.append([j,f.tolist()])
  last_token.append([j,g.tolist()])


final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/retrain_embedding_val.csv')

final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/retrain_embedding_1_val.csv')




In [ ]:
#test_seen

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoding_input=[]
last_token=[]
text=test_seen_data["text"].tolist()
id = test_seen_data["id"].tolist()

for i,j in zip(text,id):
 

  b = tokenizer(i, return_tensors='pt').to(device)
  # # c= tokenizer.tokenize(i)
  # d=tokenizer.encode(i, add_special_tokens=True)
  # #print(b.tokens)
  # print(b)
  # print(c)
  # print(d)
  output = model(**b)
  #output= model(**i)

  f=output[1][1].squeeze()[0].cpu()
  g = output[1][1].squeeze()[-1].cpu()
  encoding_input.append([j,f.tolist()])
  last_token.append([j,g.tolist()])


final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/retrain_embedding_test_seen.csv')

final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/retrain_embedding_1_test_seen.csv')

In [ ]:
#test_seen

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoding_input=[]
last_token=[]
text=test_seen_data["text"].tolist()
id = test_seen_data["id"].tolist()

for i,j in zip(text,id):
 

  b = tokenizer(i, return_tensors='pt').to(device)
  # # c= tokenizer.tokenize(i)
  # d=tokenizer.encode(i, add_special_tokens=True)
  # #print(b.tokens)
  # print(b)
  # print(c)
  # print(d)
  output = model(**b)
  #output= model(**i)

  f=output[1][1].squeeze()[0].cpu()
  g = output[1][1].squeeze()[-1].cpu()
  encoding_input.append([j,f.tolist()])
  last_token.append([j,g.tolist()])


final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/retrain_embedding_test_seen.csv')

final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/retrain_embedding_1_test_seen.csv')

In [ ]:
#test_unseen

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoding_input=[]
last_token=[]
text=test_unseen_data["text"].tolist()
id = test_unseen_data["id"].tolist()

for i,j in zip(text,id):
 

  b = tokenizer(i, return_tensors='pt').to(device)
  # # c= tokenizer.tokenize(i)
  # d=tokenizer.encode(i, add_special_tokens=True)
  # #print(b.tokens)
  # print(b)
  # print(c)
  # print(d)
  output = model(**b)
  #output= model(**i)

  f=output[1][1].squeeze()[0].cpu()
  g = output[1][1].squeeze()[-1].cpu()
  encoding_input.append([j,f.tolist()])
  last_token.append([j,g.tolist()])


final = pd.DataFrame(encoding_input)
final_copy = final.copy()
final_copy.to_csv('/content/retrain_embedding_test_unseen.csv')

final1 = pd.DataFrame(last_token)
final1_copy = final.copy()
final1_copy.to_csv('/content/retrain_embedding_1_test_unseen.csv')